# A5: Final Project Plan
Will Wright  
2019-11-14

In [ ]:
{
    "tags": [
        "hide_input",
        "hide_output"
    ]
}

import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

# I. Introduction
### Motivation/Problem Statement:
**Context:** Slay the Spire is currently the 64th most played game on Steam (out of more than 30,000 games).  It was released in late January 2019 and has received a 10/10 rating with “overwhelmingly positive” feedback.  It’s essentially a turn-based deck-building strategy and tactics game with rogue-like, procedurally-generated properties such that no two games play out the same. Winrates are generally low and few resources exist to help players improve their play.

**Gameplay:** After choosing one of three characters (Ironclad, The Silent, and Defect), players start with the same initial deck of basic cards and, through events in the game, are able to synergize toward more powerful decks as they ascend the floors of the Spire. The event types are combat (normal, elite, and boss), campfires (a heal/upgrade decision), merchants (to purchase cards/relics or remove bad cards), and "?" events, of which there are more than 100 unique decisions with random elements to help or hinder your journey. Choosing a path through these events challenges the player to balance their competing priorities to stay alive and form a solid chance of winning through building their resources. Per character, there are about 100 cards that vary in type, rarity, and upgrades. Additionally, there are over 100 relics, which are passive items that provide bonuses to gameplay. The number of permutations for deck + relic combinations alone provides for innumerable play styles, but not all are winning strategies and actual winrates are generally less than 20%.  

**Winning**: In total, there are 4 acts which take about 1.5hrs to complete and conclude with beating the final boss: the Heart of the Spire.  Only by making the correct decisions about which path, cards, and relics to pick can players form a strong enough deck to stand a chance of winning. However, even with very strong decks, the randomness of how each combat plays out means there are no guarantees of victory. Per character, once the heart is beaten, this opens up what are known as "Ascension levels"—essentially, end-game challenges that require beating the game again and gradually ramp up the difficulty per level up to the highest level: Ascension 20.  At Ascension 20, winning is very rare and some characters are more likely to win than others.  Winrates are 6%, 5%, and 2% for Ironclad, The Silent, and Defect, respectively.

**Problem:** Anyone who gets into the game and looks for resources on which resources are better soon finds https://spirelogs.com/ as the top resource.  The idea here is that you upload your game data and can see how you compare to others while providing spirelogs with data useful for determining ratings for cards and other powerups. It is worth noting that the site's data is biased toward more-hardcore players since casual players are less likely to want to upload their game data or explore optimization of their play style. Still, while the findings on this site are extremely useful to players, they lack decision-context or a sense of which final clusters of cards/relics are best.  For instance, a certain card or relic might have a high rating (i.e. is associated with a higher winrate and/or is picked more often), but be damaging for the resource-build a player currently has. My goal is to remedy this by clustering the winning deck/relic combinations into distinct archetypes and provide that context. 

**Scope of Analysis:** Because there are 3 characters, 20 Ascension levels, and not every deck wins, it wouldn't be fair to bucket all games into one group for the analysis.  Instead, I intend to limit the scope to only the winning resource combinations (cards + relics) at the highest difficulty (Ascension 20) for the Defect character (if time allows, extending this to all characters).  Because less-optimized resource combinations can win at lower Ascension levels, limiting to the highest tier should point more directly at the purest forms of winning archetypes.  I chose Defect simply because I'm currently playing through this character now and have been stuck on Ascenscion 19 for about 15 runs! With a 2% winrate at the next level, my hope is that I can use the results of this analysis to beat the final level and provide other players with insights about the winning archetypes that will help guide decisions and boost winrates. 

### Research Question
**RQ1.** How many distinct deck archetypes beat the game at Ascension 20 as Defect?

My hypothesis is that there are only about 8 ways to beat the game with these parameters, but this will not formally be tested since the methods for producing clusters (discussed below) are not statistical.


### Background and Related Work
Very little exists in the way of related work since the game was only recently released on January 23rd, 2019 and data, while public-facing, isn't easy to find without investigation.  As far as I know, Spirelogs is the only resource which exposes game data to interested players.

Still, this lack of data hasn't stopped dedicated fans from attempting to improve their gameplay with purely local data from their only personal runs.  [Learn the Spire](https://github.com/kajchang/LearnTheSpire), for example, uses local game data to train a neural net and recommend decisions in-game via an overlaying mod. The best this can do, however, is surface recommendations based on the players' past decisions, which may not be optimal.  



# II. Data Sources and Preparation

### Data Used
I wrote the creator of spirelogs, Alleji, and asked for permission to use his data.  He responded with:

_Hey!_
 
_I have an archive of runs since 1.0 right here: https://spirelogs.com/archive/_
 
_You're right in that there isn't any documentation on it, but I do get requests like yours occasionally and I'm certainly happy to share the data. Let me know if you publish your results anywhere online, I'd be interested in seeing them!_
 
_Alleji_

In these zipped files, there are over **280K completed games** in JSON format.

While no link on his page directs to the archive of data, the link provided by the creator is publically available with his blessings (as per the email). The only ethical consideration could be that individuals might be identifiable if you know their playstyle well-enough (e.g. always chooses a certain set of cards or character to play), but because identifying an individual could only give you insights about their playstyle and when they were playing, I don’t think this is a major concern. 

As for the data itself, there is one tar.gz file per month since release. Within each of these zipped folders is a folder for each day of the month and within each of those is hundreds to thousands of game .run data.  This is all in JSON format and details which decisions the users make throughout their playthroughs and their results along the way.

I’d like to point out that there isn’t a license for the data and I’m interpreting the creator’s expressed happiness to share the data as evidence that that I can use it. 

### Data Prep
As per the scope of the analysis, after extracting all JSON files, the data will be subset to only winning decks + relics for Defect in Ascension 20.  If time allows, similar analyses will be performed for Ironclad and The Silent.

Within this subset, the final card and relic combinations will be converted to a table of columns for all cards and relics with rows for each run.  A simple binary of 0 and 1 will indicate whether the card was present in the final set of resources.

### Unknown and dependencies  
Sprielogs may stop hosting the data or be turned off completely in the future. As it stands as of mid-November 2019, there are several tables in the site that are broken and Alleji makes comments such as, "Unfortunately, this page got broken by Act 4 :(".  As what appears to be a pet-project, I imagine this is something that gets occassional maintenance, but it may go down on occassion. This being the case, I’ll build my scripts to pick up the data directly from the archive (in the case that it does remain online) and if it does go down, references to the already-picked-up files in the repo will ensure reproducibility.

# III. Methodology
